In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.column import Column
from pyspark.sql.types import DoubleType

spark = SparkSession.builder.appName("silver_descuentos").getOrCreate()

BRONZE_DESCUENTOS = 'damm_bronze_des.pricing_unho.ca_crm_descuentos_test'
SILVER_DESCUENTOS = 'damm_silver_des.pricing_unho.ca_crm_descuentos_test'
SILVER_MATERIALES = 'damm_silver_des.dm.dm_material_1'


def _parse_signed_numeric(col_to_parse: Column) -> Column:
    """
    Parsea una columna de tipo string que puede tener un signo negativo al final (ej: '123.45-').
    """
    return F.when(
        col_to_parse.rlike(r'-$'),
        # Si termina con '-', lo quita, lo convierte a double y lo hace negativo.
        -F.regexp_replace(col_to_parse, r'-$', '').cast(DoubleType())
    ).otherwise(
        # En caso contrario, simplemente lo convierte a double.
        col_to_parse.cast(DoubleType())
    )

def _calculate_discount(df: DataFrame) -> DataFrame:
    """
    Calcula el 'Descuento' final basándose en reglas de negocio. Transforma los diferentes tipos de descuento (€/CAJA, EUR/BRL,..) en EUR/L o %.
    En el caso de encontrar valor de descuento positivo significa que se treata de Precio Neto, no descuento
    """
    
    # Limpia la columna 'Volumen': elimina caracteres no numéricos, reemplaza la coma por un punto y convierte el tipo.
    df = df.withColumn(
        "Volumen",
        F.regexp_replace(
            F.regexp_replace(F.col("Volumen"), r'[^0-9,]', ''),
            ",", "."
        ).cast(DoubleType())
    )

    # Asegura que CantidadBase sea numérica y no null
    df = df.withColumn(
        "CantidadBase",
        F.when(F.col("CantidadBase").isNull(), F.lit(1.0))
        .otherwise(F.col("CantidadBase").cast(DoubleType()))
    )
    
    # Asegura que Volumen no sea null ni cero
    df = df.withColumn(
        "Volumen",
        F.when((F.col("Volumen").isNull()) | (F.col("Volumen") == 0), F.lit(1.0))
        .otherwise(F.col("Volumen"))
    )

    # Define la lógica para el cálculo del descuento.
    discount_logic = F.when(
        F.col("TipoDescuento") == "Porcentaje",
        F.col("Importe")
    ).otherwise(
        # Para tipos 'Valor', calcula basándose en la unidad de medida.
        F.when(
            F.col("UnidadMedida") == "LTR",
            F.round(F.col("Importe") / F.col("Volumen"), 2)
        ).when(
            F.col("UnidadMedida") == "CS",
            F.round(F.col("Importe") / (F.col("CantidadBase") * F.col("Volumen")), 2)
        ).when(
            F.col("UnidadMedida") == "BR",
            F.round(F.col("Importe") / (F.col("CantidadBase") * F.col("Volumen")), 2)
        ).otherwise(
            # Si la UnidadMedida no coincide con ninguna, devuelve el Importe directamente
            F.col("Importe")
        )
    )

    return df.withColumn("Descuento", discount_logic)

try:
    df_bronze = spark.table(BRONZE_DESCUENTOS)
    print("Lectura completada exitosamente.")
except Exception as e:
    print(f"Error al leer la tabla {BRONZE_DESCUENTOS}. El proceso se detendrá.")
    raise e

df_cleaned = (
    df_bronze
    .withColumn("FechaInicio", F.to_date(F.col("DATAB"), "yyyyMMdd"))
    .withColumn("FechaFin", F.to_date(F.col("DATBI"), "yyyyMMdd"))
    .withColumn("KBETR", _parse_signed_numeric(F.col("KBETR")))
    .drop("DATAB", "DATBI")
)

rename_mapping = {
    "KNUM": "IdCondicion",
    "KSCHL": "ClaseCondicion",
    "KRECH": "ReglaCalculo",
    "KBETR": "Importe",
    "KONWA": "UnidadesCondicion",
    "KPEIN": "CantidadBase",
    "KMEIN": "UnidadMedida",
    "MATNR": "Material",
    "ZZESTABLEC": "EstablecimientoECC",
    "ZZDISTR": "Distribuidor",
}

# Mantiene todas las columnas, renombrando aquellas especificadas en el diccionario.
select_exprs = [
    F.col(c).alias(rename_mapping.get(c, c)) for c in df_cleaned.columns
]
df_renamed = df_cleaned.select(*select_exprs)

# busco la capcidad del envase del material
df_material = spark.table(SILVER_MATERIALES)
df_enriched = (
    df_renamed
    .join(
        df_material.select("Material", "Volumen"),
        on="Material",
        how="left"
    )
)

df_with_logic = (
    df_enriched
    .withColumn(
        "TipoDescuento",
        F.when(F.col("ReglaCalculo") == "A", "Porcentaje").otherwise("Valor")
    )
    .transform(_calculate_discount) 
)
df_silver = df_with_logic.select(
    "Material",
    "ClaseCondicion",
    "EstablecimientoECC",
    "Distribuidor",
    "FechaInicio",
    "FechaFin",
    "TipoDescuento",
    "Descuento"
)

print(f"Iniciando la escritura en la tabla Silver: {SILVER_DESCUENTOS}...")

df_silver.write \
    .mode("overwrite") \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .saveAsTable(SILVER_DESCUENTOS)

print(f"¡Proceso completado! La tabla {SILVER_DESCUENTOS} ha sido creada/actualizada correctamente.")



